# Midterm - Spring 2023

## Problem 1: Take-at-home  (45 points total)

You are applying for a position at the data science team of USDA and you are given data associated with determining appropriate parasite treatment of canines. The suggested treatment options are determined based on a **logistic regression** model that predicts if the canine is infected with a parasite. 

The data is given in the site: https://data.world/ehales/grls-parasite-study/workspace/file?filename=CBC_data.csv  and more specifically in the CBC_data.csv file. Login using you University Google account to access the data and the description that includes a paper on the study (**you dont need to read the paper to solve this problem**). Your target variable $y$ column is titled `parasite_status`. 




### Question 1 - Feature Engineering (5 points)

In this step you outline the following as potential features (this is a limited example - we can have many features as in your programming exercise below). 

Write the posterior probability expressions for logistic regression for the problem you are given to solve.

$$p(y=1| \mathbf{x}, \mathbf w)=$$ 

$$p(y=0| \mathbf{x}, \mathbf w)=$$ 

<img src="q1.png" alt="Alternative text" />



### Question 2 - Decision Boundary (5 points)

Write the expression for the decision boundary assuming that $p(y=1)=p(y=0)$. The decision boundary is the line that separates the two classes.





<img src="q2.png" alt="Alternative text" />



### Question 3 - Loss function (5 points)

Write the expression of the loss as a function of $\mathbf w$ that makes sense for you to use in this problem. 

NOTE: The loss will be a function that will include this function: 

$$\sigma(a) = \frac{1}{1+e^{-a}}$$



<img src="q3.png" alt="Alternative text" />

$$L_{CE} = $$


### Question 4 - Gradient (5 points)

Write the expression of the gradient of the loss with respect to the parameters - show all your work.



$$ \nabla_\mathbf w L_{CE} = $$

<img src="q4.png" alt="Alternative text" />

### Question 5 - Imbalanced dataset (10 points)

You are now told that in the dataset  

$$p(y=0) >> p(y=1)$$

Can you comment if the accuracy of Logistic Regression will be affected by such imbalance?



<img src="q5.png" alt="Alternative text" />


### Question 6 - SGD (15 points)

The interviewer was impressed with your answers and wants to test your programming skills. 

1. Use the dataset to train a logistic regressor that will predict the target variable $y$. 

 2. Report the harmonic mean of precision (p) and recall (r) i.e the  [metric called $F_1$ score](https://en.wikipedia.org/wiki/F-score) that is calculated as shown below using a test dataset that is 20% of each group. Plot the $F_1$ score vs the iteration number  $t$. 

$$F_1 = \frac{2}{r^{-1} + p^{-1}}$$

Your code includes hyperparameter optimization of the learning rate and mini batch size. Please learn about cross validation which is a splitting strategy for tuning models [here](https://scikit-learn.org/stable/modules/cross_validation.html).

You are allowed to use any library you want to code this problem.



In [125]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import matplotlib.pyplot as plt
import math 

df=pd.read_csv("CBC_data.csv")
df.head()
df=df.dropna()



In [133]:
y=df["PARASITE_STATUS"].replace({'Negative':0,'Positive':1}) #convert strings to values
df['SEX']=df['SEX'].replace({'Male':0,"Female":1})
df['REPRO.STATUS']=df['REPRO.STATUS'].replace({'Intact':0,'Neutered':1})
df["TYPEAREA"]=df["TYPEAREA"].replace({"Urban":1,"Suburban":0.5,"Rural":0})
#^rational behind this is that urban>suburban>rural population density, therefore more density, thus higher transmission rate?
x=df.loc[:,~df.columns.isin(['PARASITE_STATUS','SEX.REPRO','ID'])]#unimportant data for the model
#scale x values to such that no feature has greater bias due to higher values
scaler=MinMaxScaler()
x=scaler.fit_transform(x)
#split data save 30% to test
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.30,random_state=1551)
#undersample to combate abundance of y=0
undersample=RandomUnderSampler(sampling_strategy="not minority")
x_train_under, y_train_under = undersample.fit_resample(x_train, y_train)
print("class counts after undersampling:\n")
print(y_train_under.value_counts())
#fit model
classsifier=SGDClassifier(loss='log_loss',alpha=0.4,max_iter=1000, tol=0.0001)
model=classsifier.fit(x_train_under,y_train_under)
y_pred=model.predict(x_test)
# Deteremine model accuracy and goodness of fit
accuracy = accuracy_score(np.array(y_test),np.array(y_pred),normalize=True)
conf_mat = confusion_matrix(y_test,y_pred)
print("\nThe accuracy of the model is :", round(accuracy,2)*100,"%")
print("Confusion Matrix:\n", conf_mat)

precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred, average='binary')
print('Precision = ',round(precision,4),'\nRecall = ', round(recall,4), '\nF-Score = ',round(fscore,4))

print("CV score with 5 folds:",cross_val_score(model,x_train_under,y_train_under,cv=5).mean())

class counts after undersampling:

0    149
1    149
Name: PARASITE_STATUS, dtype: int64

The accuracy of the model is : 50.0 %
Confusion Matrix:
 [[408 433]
 [ 17  41]]
Precision =  0.0865 
Recall =  0.7069 
F-Score =  0.1541
CV score with 5 folds: 0.5368361581920904


### Explanation 
I used the sklearn library to fit a SDG model. Despite using the model it could not produce an accuracy greater than 65% from what ive seen. This could be due to the lack of Positive samples, despite even oversampling and undersampling. The data provided simply could not be used provide a suffecient model. 
1. Reducing the amount of parameters  produced highly volatile csv scores. 
2. Increasing alpha increases accuracy volatily, Decreasing it leaves it stagnant at about 50%
3. Without over/undersampling, the model produces a 93% accuracy but does not ever predict a positive condition(the model only guesses 0), leaving only TP's and FN's, effectively reducing f1 and precison to 0.
My hypothesis is that almost none of the data provided correlated sufficiently with Parasite_Status as shown below. Thus making it difficult for the model to fit a proper threshold. 

In [132]:
print(model.coef_)

[[-0.0461225  -0.04391969 -0.04520073 -0.03048249 -0.05144564 -0.05240167
  -0.01314039 -0.00389396 -0.01568663 -0.01039541 -0.0256525  -0.02538672]]


https://github.com/thotranexe/482-104-assignments/blob/main/Takehome/take-at-home.ipynb 
github url here